In [1]:
!pip install crypto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 7.6 MB/s eta 0:00:00


In [2]:
!pip install pycrypto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp310-cp310-linux_x86_64.whl size=498546 sha256=bc404ae9871947b7862b4c3810b8551f925d533578c647ef44dfa285666077e6
  Stored in directory: /root/.cache/pip/wheels/e8/4b/5b/b10a6fc885057b6ff9fbd5691d7e700d0a9408f80b7e6f12e0
Successfully built pycrypto


In [3]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00


In [5]:
import hashlib
import binascii
import datetime
import collections
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Cipher import PKCS1_v1_5
from collections import OrderedDict
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5

class Client:
    def __init__(self):
        rand_func = Random.new().read
        self._private_key = RSA.generate(1024, rand_func)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()

    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity
        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time': self.time})

    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')

def display_transaction(transaction):
    dict = transaction.to_dict()
    print("sender: " + dict['sender'])
    print('----- ')
    print("recipient: " + dict['recipient'])
    print('----- ')
    print("value: " + str(dict['value']))
    print('----- ')
    print("time: " + str(dict['time']))
    print('----- ')

class Block:
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.Nonce = ""

def dump_blockchain(blockchain):
    print ("Number of blocks in the chain: " + str(len(blockchain)))
    for x in range(len(blockchain)):
        block_temp = blockchain[x]
        print ("block # " + str(x))
        for transaction in block_temp.verified_transactions:
            display_transaction(transaction)
        print('---------------')
    print('=====================================')

Dinesh = Client()
t0 = Transaction(
    "Genesis",
    Dinesh.identity,
    500.0
)
block0 = Block()
block0.previous_block_hash = None
Nonce = None
block0.verified_transactions.append(t0)
TPCoins = []
TPCoins.append(block0)
dump_blockchain(TPCoins)


Number of blocks in the chain: 1
block # 0
sender: Genesis
----- 
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100af13712a8584a0f793fc6c0282b20b985e6fc28a2cce1ad1225964d125b7300100118c656dccdf1136b933c9a2ddb3d453e0491694da3f1f4e3e199825bede5f342589dadd65a721b00b95e1f064899708082e892daf759ea7c198867d1e2353027fe62927dc15d31bdf3764dccb4fb504bd490c610ed9fce4202417df94c6550203010001
----- 
value: 500.0
----- 
time: 2024-04-16 09:13:22.823095
----- 
---------------
